In [29]:
import pandas as pd

# Load the dataset
df = pd.read_excel(r"C:\Users\Shiyas Backer\Desktop\dominos\Pizza_Sale_new.xlsx")

# Convert 'order_date' to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Add a new column to represent the week number
df['week'] = df['order_date'].dt.isocalendar().week

# Check the number of unique weeks in the dataset
unique_weeks = df['week'].nunique()

print(f"Number of unique weeks: {unique_weeks}")  # Should show number of weeks (ideally 4-5 for January)

# Get the unique pizza_name_id values
unique_pizza_ids = df['pizza_name_id'].unique()

# Create an empty dictionary to store DataFrames for each pizza_name_id
pizza_dataframes = {}

# Loop through each unique pizza_name_id
for pizza_id in unique_pizza_ids:
    # Filter data for each pizza_name_id
    pizza_df = df[df['pizza_name_id'] == pizza_id]
    
    # Group by week and aggregate as needed (e.g., sum of quantity, total_price, etc.)
    weekly_grouped = pizza_df.groupby('week').agg({
        'quantity': 'sum',
        
    }).reset_index()
    
    # Store the weekly grouped data in the dictionary
    pizza_dataframes[pizza_id] = weekly_grouped

    # Optionally: Save each DataFrame to a CSV or Excel file
    # pizza_df.to_csv(f'Pizza_{pizza_id}_weekly_sales.csv', index=False)

# Example: Access the DataFrame for a specific pizza_name_id
example_pizza_id = unique_pizza_ids[0]
print(f"Data for {example_pizza_id}:\n", pizza_dataframes[example_pizza_id])

Number of unique weeks: 53
Data for hawaiian_m:
     week  quantity
0      1         3
1      2         7
2      3         8
3      4         4
4      5        11
5      6        12
6      7         7
7      8        12
8      9        10
9     10        12
10    11        10
11    12        10
12    13        10
13    14        10
14    15         9
15    16         6
16    17        11
17    18         9
18    19        15
19    20         5
20    21         6
21    22         9
22    23        16
23    24         9
24    25        10
25    26        15
26    27         6
27    28        11
28    29         4
29    30        14
30    31        10
31    32         9
32    33        10
33    34        13
34    35         5
35    36         5
36    37        15
37    38         8
38    39         6
39    40         7
40    41        10
41    42         9
42    43         8
43    44         2
44    45        13
45    46        14
46    47         9
47    48        11
48    49        10
4

In [30]:
pizza_dataframes

{'hawaiian_m':     week  quantity
 0      1         3
 1      2         7
 2      3         8
 3      4         4
 4      5        11
 5      6        12
 6      7         7
 7      8        12
 8      9        10
 9     10        12
 10    11        10
 11    12        10
 12    13        10
 13    14        10
 14    15         9
 15    16         6
 16    17        11
 17    18         9
 18    19        15
 19    20         5
 20    21         6
 21    22         9
 22    23        16
 23    24         9
 24    25        10
 25    26        15
 26    27         6
 27    28        11
 28    29         4
 29    30        14
 30    31        10
 31    32         9
 32    33        10
 33    34        13
 34    35         5
 35    36         5
 36    37        15
 37    38         8
 38    39         6
 39    40         7
 40    41        10
 41    42         9
 42    43         8
 43    44         2
 44    45        13
 45    46        14
 46    47         9
 47    48        11
 48   

In [31]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error
from pmdarima import auto_arima  # Auto ARIMA tuning
import warnings
warnings.filterwarnings("ignore")

# Assuming pizza_dataframes contains weekly dataframes for each pizza
pizza_data_mape = {}

# Loop through each pizza DataFrame
for pizza_id, pizza_df in pizza_dataframes.items():
    print(f"Training models for Pizza ID: {pizza_id}")
    
    # Ensure data is sorted by week and there's no missing data
    pizza_df = pizza_df.sort_values('week').reset_index(drop=True)
    
    # Prepare train and test sets (e.g., last 20% of weeks as test data)
    train_size = int(len(pizza_df) * 0.8)
    train_data, test_data = pizza_df[:train_size], pizza_df[train_size:]
    
    train_y, test_y = train_data['quantity'], test_data['quantity']
    
    # To store MAPE values for each model
    mape_values = {}

    ### Tuning ARIMA Model with auto_arima ###
    try:
        arima_model = auto_arima(train_y, seasonal=False, trace=False, suppress_warnings=True)
        arima_forecast = arima_model.predict(n_periods=len(test_y))
        mape_arima = mean_absolute_percentage_error(test_y, arima_forecast)
        mape_values['ARIMA'] = mape_arima
    except Exception as e:
        print(f"ARIMA failed for Pizza ID {pizza_id}: {e}")
        mape_values['ARIMA'] = None

    ### Tuning SARIMA Model with seasonal auto-arima ###
    try:
        sarima_model = auto_arima(train_y, seasonal=True, m=12, trace=False, suppress_warnings=True)
        sarima_forecast = sarima_model.predict(n_periods=len(test_y))
        mape_sarima = mean_absolute_percentage_error(test_y, sarima_forecast)
        mape_values['SARIMA'] = mape_sarima
    except Exception as e:
        print(f"SARIMA failed for Pizza ID {pizza_id}: {e}")
        mape_values['SARIMA'] = None

    ### Prophet Model (already tuned for weekly data) ###
    try:
        prophet_df = train_data.copy()
        prophet_df['ds']=pd.date_range(start='2023-01-01', periods=len(prophet_df), freq='W')
        prophet_df.rename(columns={'quantity': 'y'}, inplace=True)
        prophet_model = Prophet()
        prophet_model.fit(prophet_df)
        future = prophet_model.make_future_dataframe(periods=len(test_y), freq='W')
        prophet_forecast = prophet_model.predict(future)['yhat'].tail(len(test_y)).values
        mape_prophet = mean_absolute_percentage_error(test_y, prophet_forecast)
        mape_values['Prophet'] = mape_prophet
    except Exception as e:
        print(f"Prophet failed for Pizza ID {pizza_id}: {e}")
        mape_values['Prophet'] = None



    
    ### XGBoost Model with tuning ###
    try:
        xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=5, random_state=42)
        xgb_model.fit(np.arange(len(train_y)).reshape(-1, 1), train_y)
        xgb_forecast = xgb_model.predict(np.arange(len(train_y), len(train_y) + len(test_y)).reshape(-1, 1))
        mape_xgboost = mean_absolute_percentage_error(test_y, xgb_forecast)
        mape_values['XGBoost'] = mape_xgboost
    except Exception as e:
        print(f"XGBoost failed for Pizza ID {pizza_id}: {e}")
        mape_values['XGBoost'] = None

   
    
    # Save MAPE values for this pizza
    pizza_data_mape[pizza_id] = mape_values

# After training, print out the MAPE values
print("MAPE Values for all models and pizzas:")
for pizza_id, mape_vals in pizza_data_mape.items():
    print(f"Pizza ID: {pizza_id}, MAPE: {mape_vals}")

Training models for Pizza ID: hawaiian_m


22:56:45 - cmdstanpy - INFO - Chain [1] start processing
22:56:45 - cmdstanpy - INFO - Chain [1] done processing


Training models for Pizza ID: classic_dlx_m


22:56:47 - cmdstanpy - INFO - Chain [1] start processing
22:56:47 - cmdstanpy - INFO - Chain [1] done processing


Training models for Pizza ID: five_cheese_l


...

Training models for Pizza ID: calabrese_l


23:01:29 - cmdstanpy - INFO - Chain [1] start processing
23:01:29 - cmdstanpy - INFO - Chain [1] done processing


Training models for Pizza ID: the_greek_xxl


23:01:30 - cmdstanpy - INFO - Chain [1] start processing
23:01:30 - cmdstanpy - INFO - Chain [1] done processing


SARIMA failed for Pizza ID the_greek_xxl: shapes (4,2) and (1,) not aligned: 2 (dim 1) != 1 (dim 0)
MAPE Values for all models and pizzas:
Pizza ID: hawaiian_m, MAPE: {'ARIMA': 0.6632897165364693, 'SARIMA': 0.6632897165364693, 'Prophet': 0.6990096254059999, 'XGBoost': 0.643912721507613}
Pizza ID: classic_dlx_m, MAPE: {'ARIMA': 0.19306975944707808, 'SARIMA': 0.19306975944707808, 'Prophet': 0.1982204779381232, 'XGBoost': 0.18871484570464253}
Pizza ID: five_cheese_l, MAPE: {'ARIMA': 0.34179638271430973, 'SARIMA': 0.3487600442021338, 'Prophet': 0.2912956313300406, 'XGBoost': 0.34008121956221893}
Pizza ID: ital_supr_l, MAPE: {'ARIMA': 0.396162629776075, 'SARIMA': 0.396162629776075, 'Prophet': 0.3747531082733841, 'XGBoost': 0.38009207797514544}
Pizza ID: mexicana_m, MAPE: {'ARIMA': 0.43768100670458554, 'SARIMA': 0.4327220521733256, 'Prophet': 0.4219381629069046, 'XGBoost': 0.5338696056756939}
Pizza ID: thai_ckn_l, MAPE: {'ARIMA': 0.3403010389056703, 'SARIMA': 0.38779758150385485, 'Prophet': 

In [33]:
# Now, forecast next week's sales using the best model for each pizza
best_models = {}
for pizza_id, mape in pizza_data_mape.items():
    # Filter out None values from MAPE
    filtered_mape = {model: value for model, value in mape.items() if value is not None}
    
    if filtered_mape:  # Ensure there's at least one valid MAPE value
        best_model = min(filtered_mape, key=filtered_mape.get)  # Find the model with the lowest MAPE
        best_models[pizza_id] = best_model
    else:
        print(f"No valid MAPE values for Pizza ID {pizza_id}. Unable to select a best model.")
        best_models[pizza_id] = None  # Handle case where no models are valid

# Forecast next week's quantity using the best model
forecasts = {}
for pizza_id, model_name in best_models.items():
    if model_name is None:
        print(f"Skipping forecast for Pizza ID {pizza_id} due to lack of valid model.")
        continue  # Skip forecasting for this pizza ID

    pizza_df = pizza_dataframes[pizza_id]
    model = models[pizza_id][model_name]

    if model_name == 'ARIMA':
        # ARIMA forecast returns a series, get the first value
        forecast = model.forecast(steps=1)
        forecasts[pizza_id] = forecast.iloc[0]  # Use iloc to access the first value

    elif model_name == 'SARIMA':
        # SARIMA forecast returns a series, get the first value
        forecast = model.forecast(steps=1)
        forecasts[pizza_id] = forecast.iloc[0]  # Use iloc to access the first value

    elif model_name == 'Prophet':
        future = model.make_future_dataframe(periods=1)
        forecast = model.predict(future)
        forecasts[pizza_id] = forecast['yhat'].values[-1]  # Get the last forecast value

    elif model_name == 'XGBoost':
        next_week = pizza_df['week'].iloc[-1] + 1  # Increment week
        forecast = model.predict([[next_week]])
        forecasts[pizza_id] = forecast[0]  # XGBoost returns a list-like object, get the first value

# Display the forecasted quantities for next week
print("\nForecasted Quantities for Next Week:")
for pizza_id, forecast in forecasts.items():
    print(f"Pizza ID: {pizza_id}, Forecasted Quantity: {forecast}")


Forecasted Quantities for Next Week:
Pizza ID: hawaiian_m, Forecasted Quantity: 7.99492073059082
Pizza ID: classic_dlx_m, Forecasted Quantity: 22.993694305419922
Pizza ID: five_cheese_l, Forecasted Quantity: 23.915729429386154
Pizza ID: ital_supr_l, Forecasted Quantity: 13.481473194207378
Pizza ID: mexicana_m, Forecasted Quantity: 8.518644972018546
Pizza ID: thai_ckn_l, Forecasted Quantity: 30.969298292618063
Pizza ID: ital_supr_m, Forecasted Quantity: 8.008340835571289
Pizza ID: prsc_argla_l, Forecasted Quantity: 8.463847646085513
Pizza ID: bbq_ckn_s, Forecasted Quantity: 7.000492572784424
Pizza ID: the_greek_s, Forecasted Quantity: 5.653139303585654
Pizza ID: spinach_supr_s, Forecasted Quantity: 5.004700183868408
Pizza ID: classic_dlx_s, Forecasted Quantity: 11.004305839538574
Pizza ID: green_garden_s, Forecasted Quantity: 9.786433952644755
Pizza ID: ital_cpcllo_l, Forecasted Quantity: 13.004401206970215
Pizza ID: ital_supr_s, Forecasted Quantity: 3.3494142676248675
Pizza ID: mexica

In [34]:
best_models

{'hawaiian_m': 'XGBoost',
 'classic_dlx_m': 'XGBoost',
 'five_cheese_l': 'Prophet',
 'ital_supr_l': 'Prophet',
 'mexicana_m': 'Prophet',
 'thai_ckn_l': 'ARIMA',
 'ital_supr_m': 'XGBoost',
 'prsc_argla_l': 'Prophet',
 'bbq_ckn_s': 'XGBoost',
 'the_greek_s': 'ARIMA',
 'spinach_supr_s': 'XGBoost',
 'classic_dlx_s': 'XGBoost',
 'green_garden_s': 'Prophet',
 'ital_cpcllo_l': 'XGBoost',
 'ital_supr_s': 'Prophet',
 'mexicana_s': 'SARIMA',
 'spicy_ital_l': 'SARIMA',
 'spin_pesto_l': 'XGBoost',
 'veggie_veg_s': 'Prophet',
 'mexicana_l': 'XGBoost',
 'southw_ckn_l': 'ARIMA',
 'bbq_ckn_l': 'Prophet',
 'cali_ckn_l': 'XGBoost',
 'cali_ckn_m': 'Prophet',
 'pepperoni_l': 'Prophet',
 'cali_ckn_s': 'Prophet',
 'ckn_pesto_l': 'Prophet',
 'big_meat_s': 'ARIMA',
 'soppressata_l': 'XGBoost',
 'four_cheese_l': 'ARIMA',
 'napolitana_s': 'ARIMA',
 'calabrese_m': 'Prophet',
 'four_cheese_m': 'ARIMA',
 'ital_veggie_s': 'SARIMA',
 'mediterraneo_m': 'XGBoost',
 'peppr_salami_s': 'Prophet',
 'spinach_fet_l': 'ARIMA

In [35]:
import pandas as pd

# Create an empty DataFrame to store pizza_id and rounded forecasted quantity
forecasted_df = pd.DataFrame(columns=['pizza_id', 'quantity'])

# Loop through the forecasts and round the quantities
for pizza_id, forecast in forecasts.items():
    rounded_quantity = round(forecast)  # Round the forecasted quantity
    new_row = pd.DataFrame({'pizza_id': [pizza_id], 'quantity': [rounded_quantity]})
    forecasted_df = pd.concat([forecasted_df, new_row], ignore_index=True)

# Display the DataFrame
print(forecasted_df)

         pizza_id quantity
0      hawaiian_m        8
1   classic_dlx_m       23
2   five_cheese_l       24
3     ital_supr_l       13
4      mexicana_m        9
..            ...      ...
86    the_greek_l        4
87  soppressata_m        2
88  soppressata_s        5
89    calabrese_l        8
90  the_greek_xxl        1

[91 rows x 2 columns]


In [36]:
forecasted_df.to_csv('dominos_forecasted_pizza_quantities.csv', index=False)

In [37]:
total_quantity = forecasted_df['quantity'].sum()
total_quantity

828

In [38]:
import pandas as pd


pizza_ingredients = pd.read_excel(r"C:\Users\Shiyas Backer\Desktop\dominos\Pizza_ingredients.xlsx")

In [39]:
pizza_ingredients

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...,...
513,veggie_veg_s,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,veggie_veg_s,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,veggie_veg_s,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,veggie_veg_s,The Vegetables + Vegetables Pizza,Spinach,15.0


In [40]:
pizza_ingredients.columns

Index(['pizza_name_id', 'pizza_name', 'pizza_ingredients',
       'Items_Qty_In_Grams'],
      dtype='object')

In [41]:
missing_df=pizza_ingredients.isnull().sum().reset_index()
missing_df.columns=['column','missing values']
print(missing_df)

               column  missing values
0       pizza_name_id               0
1          pizza_name               0
2   pizza_ingredients               0
3  Items_Qty_In_Grams               4


In [42]:
# Drop rows where 'Items_Qty_In_Grams' has missing values
pizza_ingredients = pizza_ingredients.dropna(subset=['Items_Qty_In_Grams'])

In [43]:
import pandas as pd

# Merge the pizza_ingredients DataFrame with the forecast_df on pizza_id (pizza_name_id)
purchase_order_df = pd.merge(pizza_ingredients, forecasted_df, left_on='pizza_name_id', right_on='pizza_id')

#  Calculate the total ingredient requirement by multiplying forecasted quantity with Items_Qty_In_Grams
purchase_order_df['Total_Ingredient_Requirement'] = purchase_order_df['Items_Qty_In_Grams'] * purchase_order_df['quantity']

#  Group by pizza_ingredients to sum up the total amount of each ingredient required
final_purchase_order = purchase_order_df.groupby('pizza_ingredients')['Total_Ingredient_Requirement'].sum().reset_index()

#  Sort the order by ingredients for readability
final_purchase_order = final_purchase_order.sort_values(by='pizza_ingredients')


print(final_purchase_order)

          pizza_ingredients Total_Ingredient_Requirement
0              ?duja Salami                       1020.0
1             Alfredo Sauce                        235.0
2                 Anchovies                        750.0
3                Artichokes                       2095.0
4                   Arugula                        290.0
..                      ...                          ...
57       Sun-dried Tomatoes                        340.0
58  Thai Sweet Chilli Sauce                       1160.0
59                    Thyme                         55.0
60                 Tomatoes                      10300.0
61                 Zucchini                        890.0

[62 rows x 2 columns]


In [44]:
final_purchase_order.to_csv('dominos_final_purchase_order.csv', index=False)